In [42]:
import os
import openai
import numpy as np
from dotenv import load_dotenv
import re
import json
import pandas as pd
load_dotenv()
# Load your API key from an environment variable or secret management service
openai.api_key = os.getenv("API_KEY")

In [43]:
data = pd.read_csv("prompts_aug.csv")
data

,pair_id,cause,effect,prompt
0,"pair0001,",the altitude,temperature,does changing the altitude cause a change in t...
1,"pair0001,",the temperature,altitude,does changing the temperature cause a change i...
2,"pair0002,",the altitude,precipitation,does changing the altitude cause a change in p...
3,"pair0002,",the precipitation,altitude,does changing the precipitation cause a change...
4,"pair0003,",the longitude,temperature,does changing the longitude cause a change in ...
...,...,...,...,...
211,"pair0106,",the input voltage of an electric toy locomotive,its time required for one round,does changing the input voltage of an electric...
212,"pair0107,",the contrast of tilted Gabor patches,the accuracy of detection of tilt by participants,does changing the contrast of tilted Gabor pat...
213,"pair0107,",the accuracy of detection of tilt of Gabor pat...,contrast of the Gabor patches,does changing the accuracy of detection of til...
214,"pair0108,",the time for 1/6 rotation of a Stirling engine,the temperature of its heat bath,does changing the time for 1/6 rotation of a S...


# Single prompt, causal agent, gpt-3.5-turbo and gpt-4

In [49]:
# this is for gpt models using chat complete. It allows system given role
def format_message(q1w1, q1w2, q2w1, q2w2, sys_instruction = None):
    q1 = f"Changing {q1w1} causes a change in {q1w2}"
    q2 = f"Changing {q2w1} causes a change in {q2w2}"
    question = f"Which casue-and-effect relationship is more likely?\nA: {q1}.\nB: {q2}."
    requirement = "\n\nLet's work this out in a step by step way to be sure that we have the right answer. Then provide your final answer within the tags <Answer>A/B</Answer>"
    ret = [{"role": "system", "content": sys_instruction}] if sys_instruction else []
    ret.append({"role": "user", "content": question+requirement})
    
    return ret

def complete_chat(model, messages, temperature = 0):
    # role: ['system', 'assistant', 'user', 'function']
    return openai.ChatCompletion.create(model=model, messages=messages, temperature = temperature)

def generate_response_from_data(model, data):
    predictions = []

    for pair_id in data["pair_id"].unique():
        if pair_id[-2] == '0':
            print("working on pair_id " + pair_id)
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        sys_instruction = "You are a helpful assistant with causal reasoning."
        message = format_message(q1w1, q1w2, q2w1, q2w2, sys_instruction)
        
        res = complete_chat(model, message)
        print({pair_id: res})
        predictions.append({pair_id: res})
    # save the predictions to file
    with open(model+"_response_trial2.json", "w") as file:
        json.dump(predictions, file)
        
def generate_prompts(data, sys = False):
    ret = {}
    for pair_id in data["pair_id"].unique():
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        sys_instruction = "You are a helpful assistant with causal reasoning." if sys else None
        message = format_message(q1w1, q1w2, q2w1, q2w2, sys_instruction)
        ret[int(pair_id[5:-1])] = message
    return ret

### Producing the prompts, to record them in a separate file

In [30]:
result = generate_prompts(data, True)
with open("prompts_sys.json", "w") as file:
    json.dump(result, file)
result = generate_prompts(data)
with open("prompts.json", "w") as file:
    json.dump(result, file)

In [291]:
# testing message generation
sys_instruction = "You are a helpful assistant with causal reasoning."
format_message("a", "b", "b", "a", sys_instruction)

[{'role': 'user',
  'content': "Which casue-and-effect relationship is more likely?\nA: Changing a causes a change in b.\nB: Changing b causes a change in a.\n\nLet's work this out in a step by step way to be sure that we have the right answer. Then provide your final answer within the tags <Answer>A/B</Answer>"},
 {'role': 'system',
  'content': 'You are a helpful assistant with causal reasoning.'}]

In [50]:
# second experiment
# manually modified, adds trial 2 to the generated file name
model = "gpt-3.5-turbo"
generate_response_from_data(model, data)

{'pair0001,': <OpenAIObject chat.completion id=chatcmpl-7zB3i4fi82TAvQL1k66zp1v88wirS at 0x7faec174c040> JSON: {
  "id": "chatcmpl-7zB3i4fi82TAvQL1k66zp1v88wirS",
  "object": "chat.completion",
  "created": 1694814874,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze the situation step by step:\n\n1. Changing the altitude causes a change in the temperature (A): This relationship suggests that altering the altitude directly leads to a change in temperature. For example, if you increase the altitude, the temperature would change accordingly. However, this relationship seems less likely because altitude is typically influenced by factors such as geography, topography, and weather patterns, which in turn affect temperature.\n\n2. Changing temperature causes a change in altitude (B): This relationship suggests that modifyin

{'pair0006,': <OpenAIObject chat.completion id=chatcmpl-7zB4IelepwdIMEKcPMxCkS9ty7lvy at 0x7faed36c5bc0> JSON: {
  "id": "chatcmpl-7zB4IelepwdIMEKcPMxCkS9ty7lvy",
  "object": "chat.completion",
  "created": 1694814910,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, we can consider the natural progression and logical reasoning behind the variables involved.\n\nIn the case of abalone, age is a factor that naturally progresses over time. As abalone grow older, their age increases. On the other hand, shell weight is influenced by various factors such as diet, environmental conditions, and growth rate.\n\nConsidering this, it is more likely that changing the age of an abalone causes a change in the shell weight of an abalone (Option A). As abalone age, they tend to grow larger and accumulate more shell material, resulting in an increase in 

{'pair0012,': <OpenAIObject chat.completion id=chatcmpl-7zB51FOD3xLKj54hTH7btiimbBBB3 at 0x7faed36c74c0> JSON: {
  "id": "chatcmpl-7zB51FOD3xLKj54hTH7btiimbBBB3",
  "object": "chat.completion",
  "created": 1694814955,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze the two scenarios:\n\nA: Changing the age of a person causes a change in the wage per hour of a person.\nB: Changing their wage per hour causes a change in their age.\n\n1. Scenario A: Changing the age of a person causes a change in the wage per hour of a person.\nIn this scenario, the assumption is that as a person's age changes, their wage per hour also changes. This relationship seems plausible because as individuals gain more experience and skills over time, they may become more valuable to employers, leading to an increase in their wage per hour. Simi

{'pair0017,': <OpenAIObject chat.completion id=chatcmpl-7zB5eoBFsytewT1rHeZ5072oposwc at 0x7faed367fa60> JSON: {
  "id": "chatcmpl-7zB5eoBFsytewT1rHeZ5072oposwc",
  "object": "chat.completion",
  "created": 1694814994,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze the two scenarios:\n\nA: Changing the age of a person causes a change in the dividends from stocks of a person.\nB: Changing their dividends from stocks causes a change in their age.\n\n1. Scenario A: Changing the age of a person causes a change in the dividends from stocks of a person.\nIn this scenario, the assumption is that altering a person's age directly influences the dividends they receive from stocks. However, age is not typically a direct determinant of stock dividends. Dividends are usually influenced by factors such as the performance of the st

{'pair0022,': <OpenAIObject chat.completion id=chatcmpl-7zB6GTcIGb0VEQrMH52ftyFEQBdGj at 0x7fae501251c0> JSON: {
  "id": "chatcmpl-7zB6GTcIGb0VEQrMH52ftyFEQBdGj",
  "object": "chat.completion",
  "created": 1694815032,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze each scenario step by step:\n\nA: Changing the age of a person causes a change in the height of a person.\nIn this scenario, we are assuming that altering a person's age will directly impact their height. However, age and height are not causally related. Age is a measure of the number of years a person has lived, while height is determined by genetic factors, nutrition, and other environmental factors. Changing a person's age will not directly cause a change in their height. Therefore, scenario A is less likely.\n\nB: Changing their height causes a change 

...

{'pair0083,': <OpenAIObject chat.completion id=chatcmpl-7zBEAxtNoiVU7zpazYmFC4oZq3hKh at 0x7faed361ebb0> JSON: {
  "id": "chatcmpl-7zBEAxtNoiVU7zpazYmFC4oZq3hKh",
  "object": "chat.completion",
  "created": 1694815522,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship between changing temperature and local CO2 flux in Park Falls, US, we can consider the following steps:\n\n1. Review existing scientific knowledge: It is well-established that changes in temperature can influence the behavior of greenhouse gases, including CO2. Higher temperatures can enhance the release of CO2 from various sources, such as soil, vegetation, and water bodies.\n\n2. Analyze the specific context: Park Falls, US, is a location within a larger system influenced by global climate patterns. Temperature changes in Park Falls are likely to be driven by larger-scale c

{'pair0088,': <OpenAIObject chat.completion id=chatcmpl-7zBEkypm149TNd3huWh4nW68WcN9W at 0x7faed362a0c0> JSON: {
  "id": "chatcmpl-7zBEkypm149TNd3huWh4nW68WcN9W",
  "object": "chat.completion",
  "created": 1694815558,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship between changing the age of a child and changing their relative spinal bone mineral density, we can consider the natural progression of these variables.\n\n1. Age is a natural and continuous variable that progresses over time. As a child grows older, their age increases.\n\n2. Relative spinal bone mineral density is a measure of the mineral content in the bones, which can be influenced by various factors such as nutrition, physical activity, and genetics. It is unlikely that changing the relative spinal bone mineral density would directly cause a change in age.\n\nBased on th

{'pair0094,': <OpenAIObject chat.completion id=chatcmpl-7zBFOeKJsoVq5WyECpqRsEF4bMcJV at 0x7faed3657740> JSON: {
  "id": "chatcmpl-7zBFOeKJsoVq5WyECpqRsEF4bMcJV",
  "object": "chat.completion",
  "created": 1694815598,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze each scenario step by step:\n\nA: Changing the hour of day causes a change in the temperature.\n1. Changing the hour of day: If we change the hour of the day, it means we are altering the time of day, such as moving from morning to afternoon or evening.\n2. Causes a change in the temperature: If changing the hour of the day causes a change in temperature, it implies that the temperature is directly influenced by the time of day.\n\nB: Changing the temperature causes a change in the hour of day.\n1. Changing the temperature: If we change the temperature, it

{'pair0099,': <OpenAIObject chat.completion id=chatcmpl-7zBG3TyQhD63FawSV6FCh7oShGhyq at 0x7faed361c770> JSON: {
  "id": "chatcmpl-7zBG3TyQhD63FawSV6FCh7oShGhyq",
  "object": "chat.completion",
  "created": 1694815639,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze the two options:\n\nA: Changing the language test score of a student causes a change in the social-economic status of a student's family.\nB: Changing the social-economic status of their family causes a change in their language test score.\n\n1. Option A: Changing the language test score of a student causes a change in the social-economic status of a student's family.\nIt is unlikely that a student's language test score alone would directly cause a change in their family's social-economic status. Typically, a student's test score is not a determining facto

{'pair0104,': <OpenAIObject chat.completion id=chatcmpl-7zBGhQ2vCAryo0MCmrFf2WeDyyyQ8 at 0x7faed3657290> JSON: {
  "id": "chatcmpl-7zBGhQ2vCAryo0MCmrFf2WeDyyyQ8",
  "object": "chat.completion",
  "created": 1694815679,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, let's analyze the two scenarios step by step:\n\nScenario A: Changing the time interval between passing the first and the second light barrier of a ball track causes a change in the time interval between passing the third and the fourth light barrier of a ball track.\n\n1. Changing the time interval between passing the first and the second light barrier.\n2. Result: A change in the time interval between passing the third and the fourth light barrier.\n\nScenario B: Changing the time interval between passing the third and the fourth light barrier of the track causes a change 

In [318]:
# first experiment
model = "gpt-3.5-turbo"
generate_response_from_data(model, data)

{'pair0001,': <OpenAIObject chat.completion id=chatcmpl-7yrVQgnStMUdZvv6zURbGJ4DFNxxB at 0x7f84d04d4540> JSON: {
  "id": "chatcmpl-7yrVQgnStMUdZvv6zURbGJ4DFNxxB",
  "object": "chat.completion",
  "created": 1694739712,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze the situation step by step:\n\n1. Changing the altitude causes a change in the temperature (A): This relationship suggests that altering the altitude directly leads to a change in temperature. For example, if you increase the altitude, the temperature would change accordingly. However, this relationship seems less likely because altitude is typically influenced by factors such as geography, topography, and weather patterns, which in turn affect temperature.\n\n2. Changing temperature causes a change in altitude (B): This relationship suggests that modifyin

{'pair0007,': <OpenAIObject chat.completion id=chatcmpl-7yrWA2dcEtim0v1N5fJEQH2XHpBI5 at 0x7f84d04bcfe0> JSON: {
  "id": "chatcmpl-7yrWA2dcEtim0v1N5fJEQH2XHpBI5",
  "object": "chat.completion",
  "created": 1694739758,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, we need to consider the nature of the variables involved.\n\nIn option A, the cause is changing the age of an abalone, and the effect is a change in the diameter of an abalone. This relationship suggests that as the age of an abalone changes, its diameter also changes. This relationship is plausible because as abalones grow older, they tend to increase in size.\n\nIn option B, the cause is changing the diameter of an abalone, and the effect is a change in its age. This relationship suggests that as the diameter of an abalone changes, its age also changes. However, it is unli

{'pair0013,': <OpenAIObject chat.completion id=chatcmpl-7yrX4jYtvlCBOfVU67GoyApybUbOM at 0x7f84d04bf6a0> JSON: {
  "id": "chatcmpl-7yrX4jYtvlCBOfVU67GoyApybUbOM",
  "object": "chat.completion",
  "created": 1694739814,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, we can analyze the relationship between changing the displacement of a car and changing its fuel consumption.\n\nA: Changing the displacement of a car causes a change in the fuel consumption of a car.\nThis relationship suggests that altering the displacement of a car directly affects its fuel consumption. It implies that modifying the displacement, such as increasing or decreasing the engine size, would result in a corresponding change in fuel consumption.\n\nB: Changing its fuel consumption causes a change in its displacement.\nThis relationship suggests that adjusting the

{'pair0018,': <OpenAIObject chat.completion id=chatcmpl-7yrXojtnbNoKknEpKa5QjUXbmZYG0 at 0x7f84d04ac590> JSON: {
  "id": "chatcmpl-7yrXojtnbNoKknEpKa5QjUXbmZYG0",
  "object": "chat.completion",
  "created": 1694739860,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship between changing the age of a person and changing their concentration of GAG (glycosaminoglycan), we can consider the following steps:\n\n1. Review existing knowledge: It is important to consider any existing knowledge or research on the topic. In this case, we need to determine if there is any evidence or scientific understanding that supports one relationship over the other.\n\n2. Plausibility: We can assess the plausibility of each relationship based on our understanding of the variables involved. Age is a natural progression that occurs over time and is influenced by vari

{'pair0023,': <OpenAIObject chat.completion id=chatcmpl-7yrYfr3MGcq8nJKIPjlP6pW2dCMP2 at 0x7f84d02ead40> JSON: {
  "id": "chatcmpl-7yrYfr3MGcq8nJKIPjlP6pW2dCMP2",
  "object": "chat.completion",
  "created": 1694739913,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze each scenario step by step:\n\nA: Changing the age of a person causes a change in the weight of a person.\nIn this scenario, we are assuming that altering a person's age directly leads to a change in their weight. However, age and weight are not inherently linked in a cause-and-effect relationship. While age can indirectly influence weight through factors like metabolism and lifestyle changes, it is not a direct cause of weight change. Therefore, scenario A is less likely.\n\nB: Changing their weight causes a change in their age.\nIn this scenario, we are 

...

{'pair0084,': <OpenAIObject chat.completion id=chatcmpl-7yrhs0sc2wMKzspFglls84Y1O329z at 0x7f84f020b650> JSON: {
  "id": "chatcmpl-7yrhs0sc2wMKzspFglls84Y1O329z",
  "object": "chat.completion",
  "created": 1694740484,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze each scenario step by step:\n\nA: Changing employment causes a change in population.\n1. When employment changes, it can lead to people moving to different areas in search of job opportunities. This migration can cause a change in population.\n2. Additionally, changes in employment can affect birth rates and fertility rates. If employment opportunities decrease, people may delay starting a family or have fewer children, which can impact population growth.\n\nB: Changing population causes a change in employment.\n1. A change in population can influence the 

{'pair0090,': <OpenAIObject chat.completion id=chatcmpl-7yrieEVM7CbEAbDsHYDGcGpgqmtEo at 0x7f84f0274090> JSON: {
  "id": "chatcmpl-7yrieEVM7CbEAbDsHYDGcGpgqmtEo",
  "object": "chat.completion",
  "created": 1694740532,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, we need to consider the chronological order of events. \n\nIn option A, it states that changing the mass loss in October 2012 in the forest causes a change in the mass loss in April 2012 in the forest. However, this implies that an event in the future (October 2012) can cause an effect in the past (April 2012), which is not logically possible. Therefore, option A is not a likely cause-and-effect relationship.\n\nIn option B, it states that changing the mass loss in April 2012 there (presumably referring to a specific location) causes a change in the mass loss in October

{'pair0095,': <OpenAIObject chat.completion id=chatcmpl-7yrjRLPO5TKhOVfcfz6ilodBKE2id at 0x7f84f02745e0> JSON: {
  "id": "chatcmpl-7yrjRLPO5TKhOVfcfz6ilodBKE2id",
  "object": "chat.completion",
  "created": 1694740581,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship between A and B, let's analyze each statement:\n\nA: Changing the hour of day causes a change in the electricity load in a region.\nThis statement suggests that altering the hour of the day leads to a subsequent change in the electricity load. For example, if the hour of the day is shifted, it may affect the demand for electricity, resulting in a change in the electricity load.\n\nB: Changing the electricity load in a region causes a change in the hour of day.\nThis statement implies that modifying the electricity load in a region leads to a subsequent change in the hour of t

{'pair0100,': <OpenAIObject chat.completion id=chatcmpl-7yrk9XPoGj875rRVtLkhZcicTg53t at 0x7f84f02ed3a0> JSON: {
  "id": "chatcmpl-7yrk9XPoGj875rRVtLkhZcicTg53t",
  "object": "chat.completion",
  "created": 1694740625,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine which cause-and-effect relationship is more likely, let's analyze the two scenarios:\n\nA: Changing the cycle time of a CPU causes a change in the performance of a CPU on a benchmark mix.\nB: Changing its performance on a benchmark mix causes a change in its cycle time.\n\n1. Scenario A: Changing the cycle time of a CPU causes a change in the performance of a CPU on a benchmark mix.\nIn this scenario, we are assuming that altering the cycle time of a CPU directly affects its performance on a benchmark mix. This relationship seems plausible because the cycle time of a CPU can impact its processing speed and efficiency, wh

{'pair0105,': <OpenAIObject chat.completion id=chatcmpl-7yrkxqsInZ8S8mxr8QSqkfmYCniTf at 0x7f84d04bc2c0> JSON: {
  "id": "chatcmpl-7yrkxqsInZ8S8mxr8QSqkfmYCniTf",
  "object": "chat.completion",
  "created": 1694740675,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, let's analyze the situation step by step:\n\nA: Changing the pixel vector of a patch causes a change in the total brightness of the screen.\nThis relationship suggests that modifying the pixel vector of a patch directly influences the overall brightness of the screen. It implies that adjusting the individual pixels leads to a change in the screen's brightness.\n\nB: Changing the total brightness of the screen causes a change in the pixel vector of a patch.\nThis relationship suggests that altering the overall brightness of the screen affects the pixel vector of a patch. It i

In [285]:
model = "gpt-4"
generate_response_from_data(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


# Single prompt, no causal agent, gpt-3.5-turbo and gpt-4

In [55]:

def generate_response_nosystem(model, data):
    predictions = []

    for pair_id in data["pair_id"].unique():
        if pair_id[-2] == '0':
            print("working on pair_id " + pair_id)
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        message = format_message(q1w1, q1w2, q2w1, q2w2)
        
        res = complete_chat(model, message)
        print({pair_id: res})
        predictions.append({pair_id: res})
    # save the predictions to file
    with open(model+"_response_nosys.json", "w") as file:
        json.dump(predictions, file)

In [11]:
model = "gpt-3.5-turbo"
generate_response_nosystem(model, data)

{'pair0001,': <OpenAIObject chat.completion id=chatcmpl-7z6HzHATzsuKcezTTU9zfywBZ0GKf at 0x7fae5001d8a0> JSON: {
  "id": "chatcmpl-7z6HzHATzsuKcezTTU9zfywBZ0GKf",
  "object": "chat.completion",
  "created": 1694796539,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship between changing altitude and changing temperature, we can consider the following steps:\n\n1. Identify the variables: The variables in this scenario are altitude and temperature.\n\n2. Determine the possible cause: In this case, changing altitude and changing temperature are both potential causes.\n\n3. Analyze the relationship: Changing altitude could potentially cause a change in temperature, as higher altitudes generally have lower temperatures due to the decrease in air pressure and the associated decrease in heat. On the other hand, changing temperature could also cause a change in

{'pair0006,': <OpenAIObject chat.completion id=chatcmpl-7z6IcXaBtu4ohKTVTWegwzNb6wFDf at 0x7fae5001f920> JSON: {
  "id": "chatcmpl-7z6IcXaBtu4ohKTVTWegwzNb6wFDf",
  "object": "chat.completion",
  "created": 1694796578,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship, we need to consider the logical sequence of events. \n\nIn the case of option A, it suggests that changing the age of an abalone causes a change in the shell weight of an abalone. This implies that the age of the abalone is the cause, and the shell weight is the effect. It is reasonable to assume that as an abalone grows older, its shell weight may increase due to factors such as accumulation of calcium carbonate.\n\nIn the case of option B, it suggests that changing the shell weight causes a change in the age of an abalone. This implies that the shell weight is the cause, and the age i

{'pair0012,': <OpenAIObject chat.completion id=chatcmpl-7z6JPpqtPLzBaUmmeUFtDWzHU4mM5 at 0x7fae50034bd0> JSON: {
  "id": "chatcmpl-7z6JPpqtPLzBaUmmeUFtDWzHU4mM5",
  "object": "chat.completion",
  "created": 1694796627,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, we need to consider the logical sequence of events and the plausibility of each scenario.\n\nA: Changing the age of a person causes a change in the wage per hour of a person.\nThis relationship suggests that as a person's age changes, their wage per hour also changes. It is plausible that as individuals gain more experience and skills with age, their wage per hour may increase. However, it is also possible that other factors such as education, job performance, or market demand for certain skills may have a more significant impact on wage changes. Therefore, this relationship

{'pair0017,': <OpenAIObject chat.completion id=chatcmpl-7z6K89sDJlx3YB0Fo7wHwEHBMwsyT at 0x7fae703ca930> JSON: {
  "id": "chatcmpl-7z6K89sDJlx3YB0Fo7wHwEHBMwsyT",
  "object": "chat.completion",
  "created": 1694796672,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship, we need to consider the logical sequence of events. \n\nStep 1: Changing the age of a person causes a change in the dividends from stocks of a person.\nThis statement suggests that altering a person's age directly leads to a change in their dividends from stocks. However, it is unlikely that changing someone's age would have a direct impact on their investment returns. Age is not typically a determining factor in the dividends received from stocks.\n\nStep 2: Changing their dividends from stocks causes a change in their age.\nThis statement implies that modifying a person's dividends fr

{'pair0023,': <OpenAIObject chat.completion id=chatcmpl-7z6Kr2IFrhaZ1jsrDwlRRyE5FeuWQ at 0x7fae704474c0> JSON: {
  "id": "chatcmpl-7z6Kr2IFrhaZ1jsrDwlRRyE5FeuWQ",
  "object": "chat.completion",
  "created": 1694796717,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship, we need to consider the logical sequence of events and the plausibility of each scenario.\n\nA: Changing the age of a person causes a change in the weight of a person.\nThis relationship seems unlikely because age and weight are not directly related. While weight can change over time due to various factors such as diet, exercise, and health conditions, age itself does not cause weight changes.\n\nB: Changing their weight causes a change in their age.\nThis relationship is also implausible. Age is a fixed measure of time and cannot be influenced or changed by weight fluctuations. Weight 

...

{'pair0087,': <OpenAIObject chat.completion id=chatcmpl-7z6TL6q1hSnQ6eJ6FHEiVYGaDs3fx at 0x7fae703eae30> JSON: {
  "id": "chatcmpl-7z6TL6q1hSnQ6eJ6FHEiVYGaDs3fx",
  "object": "chat.completion",
  "created": 1694797243,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship between changing temperature and the total amount of snow, we can consider the following:\n\n1. Changing the temperature causes a change in the total amount of snow: This relationship suggests that altering the temperature leads to a subsequent change in the total amount of snow. For example, if the temperature increases, it may result in less snowfall, while a decrease in temperature could lead to more snowfall.\n\n2. Changing the total amount of snow causes a change in the temperature: This relationship implies that modifying the total amount of snow has an impact on the temperature. F

{'pair0093,': <OpenAIObject chat.completion id=chatcmpl-7z6U3S6osPJklRk4kz3YcaDLABR6F at 0x7fae7042f1a0> JSON: {
  "id": "chatcmpl-7z6U3S6osPJklRk4kz3YcaDLABR6F",
  "object": "chat.completion",
  "created": 1694797287,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the more likely cause-and-effect relationship, we can analyze the steps involved in each scenario:\n\nA: Changing the precipitation causes a change in the runoff.\n1. Changing the precipitation: This refers to altering the amount or pattern of rainfall in a specific area.\n2. Causes a change in the runoff: Runoff is the water that flows over the land surface and into rivers, lakes, or other bodies of water.\n\nB: Changing the runoff in river catchment areas causes a change in the precipitation in river catchment areas.\n1. Changing the runoff: This involves modifying the amount or flow of water that runs off the land sur

{'pair0098,': <OpenAIObject chat.completion id=chatcmpl-7z6UlnwXk1WretfqfmZErlEKHjXFp at 0x7fae500856c0> JSON: {
  "id": "chatcmpl-7z6UlnwXk1WretfqfmZErlEKHjXFp",
  "object": "chat.completion",
  "created": 1694797331,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship, let's consider the sequence of events in each scenario:\n\nA: Changing a ball's speed at the beginning of a ball track with an acceleration zone causes a change in a ball's speed at the end of a ball track with an acceleration zone.\n\n1. The ball's speed is changed at the beginning of the track.\n2. The ball travels through the acceleration zone.\n3. The ball's speed is observed at the end of the track.\n\nB: Changing its speed at the end of the track causes a change in its speed at the beginning of the track.\n\n1. The ball's speed is observed at the end of the track.\n2. The ball's s

{'pair0103,': <OpenAIObject chat.completion id=chatcmpl-7z6VVCbh38Xt0Pi0ELZJgoNwjn3Ro at 0x7fae50087fb0> JSON: {
  "id": "chatcmpl-7z6VVCbh38Xt0Pi0ELZJgoNwjn3Ro",
  "object": "chat.completion",
  "created": 1694797377,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship, we need to consider the logical sequence of events. \n\nIn option A, it states that changing the initial position of the ball in a ball track causes a change in the time interval between passing the third and the fourth light barrier of the track. This means that altering the starting position of the ball leads to a difference in the time it takes for the ball to pass the third and fourth light barriers.\n\nIn option B, it states that changing the time interval between passing the third and the fourth light barrier of the track causes a change in the initial position of the ball in the 

{'pair0108,': <OpenAIObject chat.completion id=chatcmpl-7z6WFH2ieJwJTkTYHCp1oV2lhqq2X at 0x7fae703ebf60> JSON: {
  "id": "chatcmpl-7z6WFH2ieJwJTkTYHCp1oV2lhqq2X",
  "object": "chat.completion",
  "created": 1694797423,
  "model": "gpt-3.5-turbo-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "To determine the cause-and-effect relationship, let's analyze the two statements:\n\nA: Changing the time for 1/6 rotation of a Stirling engine causes a change in the temperature of the heat bath of a Stirling engine.\nB: Changing the temperature of its heat bath causes a change in time it takes for 1/6 rotation.\n\nStep 1: Identify the variables involved.\nIn statement A, the variables are the time for 1/6 rotation and the temperature of the heat bath.\nIn statement B, the variables are the temperature of the heat bath and the time it takes for 1/6 rotation.\n\nStep 2: Determine the direction of causality.\nIn statement A, it sugges

In [58]:
model = "gpt-4"
generate_response_nosystem(model, data)

{'pair0001,': <OpenAIObject chat.completion id=chatcmpl-7zC2Vh8DaJ1mKsFQIoTL9ND1hguNT at 0x7fae480e0680> JSON: {
  "id": "chatcmpl-7zC2Vh8DaJ1mKsFQIoTL9ND1hguNT",
  "object": "chat.completion",
  "created": 1694818643,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The relationship between altitude and temperature is that as you go higher in altitude, the temperature generally decreases. This is due to the fact that the atmosphere becomes thinner with increasing altitude and thus is less able to absorb and retain heat. \n\nOn the other hand, changing the temperature does not cause a change in altitude. For example, if it gets colder or hotter where you are, you don't suddenly find yourself at a higher or lower altitude. \n\nSo, the cause-and-effect relationship that is more likely is: Changing the altitude causes a change in the temperature.\n\n<Answer>A</Answer>"
      },
      "finish_reason": "stop"

{'pair0009,': <OpenAIObject chat.completion id=chatcmpl-7zC3Xsrd4dYY03GMQglo4ppIgifZL at 0x7fae600424d0> JSON: {
  "id": "chatcmpl-7zC3Xsrd4dYY03GMQglo4ppIgifZL",
  "object": "chat.completion",
  "created": 1694818707,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Step 1: Consider the first statement - Changing the age of an abalone causes a change in the whole weight of an abalone. This statement implies that as an abalone grows older, its weight changes. This is a plausible cause-and-effect relationship because as living organisms grow older, they tend to grow in size and weight.\n\nStep 2: Consider the second statement - Changing its whole weight causes a change in its age. This statement implies that if you change the weight of an abalone, its age will change. This is not a plausible cause-and-effect relationship because the age of a living organism is determined by the passage of time, not by its

{'pair0017,': <OpenAIObject chat.completion id=chatcmpl-7zC4nqYmuuaH7LQQ3hytRKJj8YZml at 0x7fae480ee430> JSON: {
  "id": "chatcmpl-7zC4nqYmuuaH7LQQ3hytRKJj8YZml",
  "object": "chat.completion",
  "created": 1694818785,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Firstly, let's understand what each statement is saying:\n\nA: Changing the age of a person causes a change in the dividends from stocks of a person. This implies that as a person gets older, the dividends they receive from their stocks change. This could be possible as with age, a person might invest more, diversify their portfolio or make different investment decisions.\n\nB: Changing their dividends from stocks causes a change in their age. This implies that if the dividends a person receives from their stocks change, their age changes. This is not possible as age is a factor of time and cannot be influenced by financial changes.\n\nThere

{'pair0024,': <OpenAIObject chat.completion id=chatcmpl-7zC5rNQYelBdvZ3dLdwU6YRH9JHMS at 0x7fae480e09f0> JSON: {
  "id": "chatcmpl-7zC5rNQYelBdvZ3dLdwU6YRH9JHMS",
  "object": "chat.completion",
  "created": 1694818851,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The cause-and-effect relationship that is more likely is A: Changing the age of a person causes a change in the heart rate of a person. As a person ages, their heart rate can change due to various factors such as health conditions, physical fitness, and lifestyle.\n\n<Answer>A</Answer>"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 85,
    "completion_tokens": 60,
    "total_tokens": 145
  }
}}
{'pair0025,': <OpenAIObject chat.completion id=chatcmpl-7zC5v09vkUQd7Se2amK5W0KnkhtN3 at 0x7fae600c1e40> JSON: {
  "id": "chatcmpl-7zC5v09vkUQd7Se2amK5W0KnkhtN3",
  "object": "chat.completion",
  "created": 169481

{'pair0033,': <OpenAIObject chat.completion id=chatcmpl-7zC6mdMrGIbqRIH283dROADl2qmMh at 0x7fae600df740> JSON: {
  "id": "chatcmpl-7zC6mdMrGIbqRIH283dROADl2qmMh",
  "object": "chat.completion",
  "created": 1694818908,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Mean corpuscular volume (MCV) is a measure of the average size of red blood cells. It can be affected by various factors, including alcohol consumption. On the other hand, it's unlikely that changes in MCV would directly influence a person's alcohol consumption, as MCV is not a factor that people typically consider when deciding whether or not to drink alcohol. \n\n<Answer>A</Answer>"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 95,
    "completion_tokens": 81,
    "total_tokens": 176
  }
}}
{'pair0034,': <OpenAIObject chat.completion id=chatcmpl-7zC6tnXjv6M3xUGPQ3uMM0Ho7geF6 at 0x7fae60055fd0> JSON: {


...

{'pair0073,': <OpenAIObject chat.completion id=chatcmpl-7zCBmCAqJoZ3BG5ALHoP905qj8e4s at 0x7fae48094e00> JSON: {
  "id": "chatcmpl-7zCBmCAqJoZ3BG5ALHoP905qj8e4s",
  "object": "chat.completion",
  "created": 1694819218,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "CO2 emissions are primarily caused by burning fossil fuels, such as coal, oil, and gas. When we use energy, especially in forms like electricity and transportation, we often burn these fossil fuels, which releases CO2 into the atmosphere. Therefore, if we change our energy use, such as by using less electricity or driving less, we would likely change our CO2 emissions.\n\nOn the other hand, changing CO2 emissions doesn't necessarily cause a change in energy use. For example, we could reduce CO2 emissions by switching to renewable energy sources, like wind or solar power, but this wouldn't necessarily mean we're using less energy. We could al

{'pair0079,': <OpenAIObject chat.completion id=chatcmpl-7zCCw8lJB9SoAir8PIdy4VmZwINt3 at 0x7fae600ef1a0> JSON: {
  "id": "chatcmpl-7zCCw8lJB9SoAir8PIdy4VmZwINt3",
  "object": "chat.completion",
  "created": 1694819290,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "Direct photosynthetic photon flux density (PPFD) refers to the amount of light available for photosynthesis, which is a crucial process for plant growth and, by extension, ecosystem productivity. \n\nOn the other hand, net ecosystem productivity (NEP) is a measure of the total amount of carbon sequestered by an ecosystem, which is largely dependent on the rate of photosynthesis in plants.\n\nTherefore, it is more likely that changing the direct photosynthetic photon flux density, or the amount of light available for photosynthesis, would cause a change in net ecosystem productivity. If there is more light available for photosynthesis, plants

{'pair0085,': <OpenAIObject chat.completion id=chatcmpl-7zCE6xCTYjPHAKRXLZmdrJipAyDXT at 0x7fae48099440> JSON: {
  "id": "chatcmpl-7zCE6xCTYjPHAKRXLZmdrJipAyDXT",
  "object": "chat.completion",
  "created": 1694819362,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The cause-and-effect relationship in statement A is more likely. The time of measurement could refer to the time of day the milk is collected, which could potentially affect the protein content due to natural biological processes in the cow. However, changing the protein content of the milk would not have any effect on the time of measurement.\n\n<Answer>A</Answer>"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 83,
    "completion_tokens": 70,
    "total_tokens": 153
  }
}}
{'pair0086,': <OpenAIObject chat.completion id=chatcmpl-7zCEA22FocAcUDbnMkcgh6ZNeRFvE at 0x7faec1704040> JSON: {
  "id": "chatcmpl-7

{'pair0093,': <OpenAIObject chat.completion id=chatcmpl-7zCF5Yglsa80l4H8s7LI89fvfl4oy at 0x7fae48137470> JSON: {
  "id": "chatcmpl-7zCF5Yglsa80l4H8s7LI89fvfl4oy",
  "object": "chat.completion",
  "created": 1694819423,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The cause-and-effect relationship in statement A is more likely. Precipitation is a natural weather event that contributes to the amount of runoff in an area. When it rains, the water that does not get absorbed into the ground becomes runoff. Therefore, if the amount of precipitation changes, it is likely that the amount of runoff will also change.\n\nOn the other hand, the relationship in statement B is less likely. Runoff is the result of precipitation and other factors like the type of ground surface, slope of the land, and vegetation. While it's true that runoff can contribute to the overall water cycle, which includes evaporation and pr

{'pair0101,': <OpenAIObject chat.completion id=chatcmpl-7zCG0wkjmjgqPAG5a1ur1PKG5NF1j at 0x7faea10c3420> JSON: {
  "id": "chatcmpl-7zCG0wkjmjgqPAG5a1ur1PKG5NF1j",
  "object": "chat.completion",
  "created": 1694819480,
  "model": "gpt-4-0613",
  "choices": [
    {
      "index": 0,
      "message": {
        "role": "assistant",
        "content": "The grey value of a pixel refers to its intensity, which directly affects how bright or dark the pixel appears. Therefore, changing the grey value of a pixel would indeed cause a change in the brightness of the screen. On the other hand, changing the brightness of the screen is a global adjustment that affects all pixels equally, but it does not change the individual grey values of the pixels.\n\n<Answer>A</Answer>"
      },
      "finish_reason": "stop"
    }
  ],
  "usage": {
    "prompt_tokens": 91,
    "completion_tokens": 83,
    "total_tokens": 174
  }
}}
{'pair0102,': <OpenAIObject chat.completion id=chatcmpl-7zCG5pwjWqgwojpNMUX6v4SHn

# Double prompt, as causal agent, gpt-3.5-turbo

In [52]:
# double prompt, no causal agent, gpt-3.5-turbo
def generate_response_withsystem_double_prompt(model, data):
    ret = {}
    sys_instruction = "You are a helpful assistant with causal reasoning."
    predictions = []
    
    for pair_id in data["pair_id"].unique():
        print("working on pair_id " + pair_id)
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        
        q1 = f"Does changing {q1w1} cause a change in {q1w2}? Please answer in a single word: Yes or No."
        q2 = f"Does changing {q2w1} cause a change in {q2w2}? Please answer in a single word: Yes or No."
        
        message1 = [{"role": "system", "content": sys_instruction}, {"role": "user", "content": q1}]
        message2 = [{"role": "system", "content": sys_instruction}, {"role": "user", "content": q2}]
        
        res1 = complete_chat(model, message1)
        res2 = complete_chat(model, message2)
        
        predictions.append({pair_id+"_res1": res1})
        predictions.append({pair_id+"_res2": res2})
        
        ret[pair_id[5:-1]+"_res1"] = message1
        ret[pair_id[5:-1]+"_res2"] = message2
        
    # save the predictions to file
    with open(model+"_response_withsys_doubleprompt.json", "w") as file:
        json.dump(predictions, file)
    
    with open("double_prompts_withsystem.json", "w") as file:
        json.dump(ret, file)

In [54]:
model = "gpt-3.5-turbo"
generate_response_withsystem_double_prompt(model, data)

working on pair_id pair0001,
working on pair_id pair0002,
working on pair_id pair0003,
working on pair_id pair0004,
working on pair_id pair0005,
working on pair_id pair0006,
working on pair_id pair0007,
working on pair_id pair0008,
working on pair_id pair0009,
working on pair_id pair0010,
working on pair_id pair0011,
working on pair_id pair0012,
working on pair_id pair0013,
working on pair_id pair0014,
working on pair_id pair0015,
working on pair_id pair0016,
working on pair_id pair0017,
working on pair_id pair0018,
working on pair_id pair0019,
working on pair_id pair0020,
working on pair_id pair0021,
working on pair_id pair0022,
working on pair_id pair0023,
working on pair_id pair0024,
working on pair_id pair0025,
working on pair_id pair0026,
working on pair_id pair0027,
working on pair_id pair0028,
working on pair_id pair0029,
working on pair_id pair0030,
working on pair_id pair0031,
working on pair_id pair0032,
working on pair_id pair0033,
working on pair_id pair0034,
working on pai

# Double prompt, no causal agent, gpt-3.5-turbo

In [35]:
# double prompt, no causal agent, gpt-3.5-turbo
def generate_response_nosystem_double_prompt(model, data):
    predictions = []
    
    for pair_id in data["pair_id"].unique():
        if pair_id[-2] == '0':
            print("working on pair_id " + pair_id)
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        
        q1 = f"Does changing {q1w1} cause a change in {q1w2}? Please answer in a single word: Yes or No."
        q2 = f"Does changing {q2w1} cause a change in {q2w2}? Please answer in a single word: Yes or No."
        
        message1 = [{"role": "user", "content": q1}]
        message2 = [{"role": "user", "content": q2}]
        
        res1 = complete_chat(model, message1)
        res2 = complete_chat(model, message2)
        
        predictions.append({pair_id+"_res1": res1})
        predictions.append({pair_id+"_res2": res2})
        
    # save the predictions to file
    with open(model+"_response_nosys_doubleprompt.json", "w") as file:
        json.dump(predictions, file)
    
def generate_double_prompts(data):
    ret = {}
    for pair_id in data["pair_id"].unique():
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        
        q1 = f"Does changing {q1w1} cause a change in {q1w2}? Please answer in a single word: Yes or No."
        q2 = f"Does changing {q2w1} cause a change in {q2w2}? Please answer in a single word: Yes or No."
        
        message1 = [{"role": "user", "content": q1}]
        message2 = [{"role": "user", "content": q2}]
        
        ret[pair_id[5:-1]+"_res1"] = message1
        ret[pair_id[5:-1]+"_res2"] = message2
    return ret

### Generating the prompts, to record them in a separate file

In [41]:
result = generate_double_prompts(data)
with open("double_prompts.json", "w") as file:
    json.dump(result, file)

### Generating the actual response

In [18]:
model = "gpt-3.5-turbo"
generate_response_nosystem_double_prompt(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


# Double prompt, text-davinci-003 and below

In [37]:
# This is for earlier models using complete, not chat complete
def complete(model, w1,w2):
    q = f"Does changing {w1} cause a change in {w2}? Please answer in a single word: Yes or No."
    return openai.Completion.create(
        model=model,
        prompt=q,
        max_tokens=7,
        temperature=0
    )

def generate_one_way_response_from_data(model, data):
    predictions = []

    for pair_id in data["pair_id"].unique():
        if pair_id[-2] == '0':
            print("working on pair_id " + pair_id)
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]

        res1 = complete(model, q1w1, q1w2)
        res2 = complete(model, q2w1, q2w2)

        predictions.append({pair_id+"_res1": res1})
        predictions.append({pair_id+"_res2": res2})
        
    # save the predictions to file
    with open(model+"_response.json", "w") as file:
        json.dump(predictions, file)
        
def generate_double_prompts2(data):
    ret = {}
    for pair_id in data["pair_id"].unique():
        pair_data = data.query(f"pair_id == '{pair_id}'")
        q1w1 = pair_data["cause"].values[0]
        q1w2 = pair_data["cause"].values[1]
        q2w1 = pair_data["effect"].values[0]
        q2w2 = pair_data["effect"].values[1]
        
        q1 = f"Does changing {q1w1} cause a change in {q1w2}? Please answer in a single word: Yes or No."
        q2 = f"Does changing {q2w1} cause a change in {q2w2}? Please answer in a single word: Yes or No."
        
        ret[pair_id[5:-1]+"_res1"] = q1
        ret[pair_id[5:-1]+"_res2"] = q2
    return ret

In [40]:
result = generate_double_prompts2(data)
with open("double_prompts2.json", "w") as file:
    json.dump(result, file)

In [306]:
model = "text-davinci-003"
generate_one_way_response_from_data(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


In [311]:
model = "text-davinci-002"
generate_one_way_response_from_data(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


In [312]:
model = "text-davinci-001"
generate_one_way_response_from_data(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


In [7]:
model = "text-babbage-001"
generate_one_way_response_from_data(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


In [8]:
model = "text-ada-001"
generate_one_way_response_from_data(model, data)

working on pair_id pair0010,
working on pair_id pair0020,
working on pair_id pair0030,
working on pair_id pair0040,
working on pair_id pair0050,
working on pair_id pair0060,
working on pair_id pair0070,
working on pair_id pair0080,
working on pair_id pair0090,
working on pair_id pair0100,


# A result with temperature=1, single prompt, causal agent, gpt-3.5-turbo
## Skip to bottom: acc = 0.89

In [168]:
with open("data.json", "r") as file:
    loaded_data = json.load(file)

import re

def extract_answers(s):
    # Compile regex patterns
    pairid_pattern = re.compile(r"\{'pair0(\d{3}),")
    answer_pattern = re.compile(r'<Answer>(.*?)</Answer>')
    
    # Search for pairid
    pairid_match = pairid_pattern.search(s)
    pairid = pairid_match.group(1) if pairid_match else None
    
    # Search for answer
    ret = answer_pattern.findall(s)
    
    # Error handling for answer
    if len(ret) > 1:
        return pairid, "Error: more than one"
    elif len(ret) == 0:
        return pairid, "Error: empty"
    
    return pairid, ret[0]


In [204]:
labels = pd.read_csv("results.txt", delimiter = ",").loc[:,["pairid", " varA","varB","dataset","groundtruth"]]
labels['pairid'] = labels['pairid'].str.extract('(\d{4})').astype(int) # convert pairid to int
labels["groundtruth"] = labels["groundtruth"].str.replace(r'\s+', '', regex=True) # remove white space in labels

weights = pd.read_csv("pairmeta.txt", delimiter = " ", header = None).iloc[:,[0,5]]
weights.rename(columns = {0:"pairid", 5:"weight"}, inplace=True)
weights['pairid'] = weights['pairid'].astype(int) # convert pairid to int

In [205]:
labels.tail()

,pairid,varA,varB,dataset,groundtruth
103,104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->
104,105,pixel vector of a patch,total brightness at the screen,D. Janzing,->
105,106,time required for one round,voltage,D. Janzing,<-
106,107,strength of contrast,answer correct or not,"Schuett, edited by D. Janzing",->
107,108,time for 1/6 rotation,temperature,D. Janzing,<-


In [189]:
weights.tail()

,pairid,weight
103,104,0.2
104,105,0.5
105,106,1.0
106,107,1.0
107,108,1.0


In [227]:
merged_df = labels.merge(weights, on='pairid')
merged_df["gpt-3.5-turbo"] = ""
merged_df.head()

,pairid,varA,varB,dataset,groundtruth,weight,gpt-3.5-turbo
0,1,Altitude,Temperature,DWD,->,0.166,
1,2,Altitude,Precipitation,DWD,->,0.166,
2,3,Longitude,Temperature,DWD,->,0.167,
3,4,Altitude,Sunshine hours,DWD,->,0.166,
4,5,Age,Length,Abalone,->,0.143,


In [268]:
def add_prediction(merged_df, loaded_data, model):
    merged_df.set_index('pairid', inplace=True)
    for response in loaded_data:
        pairid, answer = extract_answers(str(response))
        print(pairid, answer)
        if answer != 'A/B' and answer[0] != 'E' and (answer[0]=='A' or answer[0]=='B'):
            answer = answer[0]
        answer = '->' if answer == 'A' else '<-' if answer == 'B' else answer
        merged_df.loc[int(pairid),model] = answer
        print("Converted:", answer, "\n")
    return merged_df

In [229]:
results = add_prediction(merged_df, loaded_data, "gpt-3.5-turbo")

001 A: Changing the altitude causes a change in the temperature.
Converted: -> 

002 B
Converted: <- 

003 A
Converted: -> 

004 A
Converted: -> 

005 A
Converted: -> 

006 A: Changing the age of an abalone causes a change in the shell weight of an abalone.
Converted: -> 

007 A
Converted: -> 

008 A
Converted: -> 

009 A
Converted: -> 

010 A
Converted: -> 

011 A
Converted: -> 

012 B
Converted: <- 

013 A
Converted: -> 

014 A
Converted: -> 

015 A
Converted: -> 

016 A
Converted: -> 

017 A
Converted: -> 

018 A
Converted: -> 

019 A
Converted: -> 

020 A
Converted: -> 

021 B
Converted: <- 

022 A
Converted: -> 

023 A
Converted: -> 

024 A
Converted: -> 

025 A
Converted: -> 

026 A
Converted: -> 

027 A: Changing the amount of fly ash in concrete causes a change in the compressive strength of concrete.
Converted: -> 

028 A
Converted: -> 

029 A
Converted: -> 

030 A
Converted: -> 

031 A
Converted: -> 

032 A
Converted: -> 

033 A
Converted: -> 

034 A
Converted: -> 

035 A
Con

In [230]:
results

,varA,varB,dataset,groundtruth,weight,gpt-3.5-turbo
pairid,,,,,,
1,Altitude,Temperature,DWD,->,0.166,->
2,Altitude,Precipitation,DWD,->,0.166,<-
3,Longitude,Temperature,DWD,->,0.167,->
4,Altitude,Sunshine hours,DWD,->,0.166,->
5,Age,Length,Abalone,->,0.143,->
...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,->
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,->
106,time required for one round,voltage,D. Janzing,<-,1.000,<-


In [236]:
results["correct"] = np.where(results["groundtruth"] == results["gpt-3.5-turbo"], 1, 0)
results

,varA,varB,dataset,groundtruth,weight,gpt-3.5-turbo,correct
pairid,,,,,,,
1,Altitude,Temperature,DWD,->,0.166,->,1
2,Altitude,Precipitation,DWD,->,0.166,<-,0
3,Longitude,Temperature,DWD,->,0.167,->,1
4,Altitude,Sunshine hours,DWD,->,0.166,->,1
5,Age,Length,Abalone,->,0.143,->,1
...,...,...,...,...,...,...,...
104,time for passing 1. segment,time for passing 2. segment,D. Janzing,->,0.200,->,1
105,pixel vector of a patch,total brightness at the screen,D. Janzing,->,0.500,->,1
106,time required for one round,voltage,D. Janzing,<-,1.000,<-,1


In [239]:
score = sum(results["correct"]*results["weight"]) / results["weight"].sum()

In [240]:
score

0.8910454152659504